In [25]:
import os
import json
import requests

In [26]:
leagues = [48, 21114]

In [27]:
def create_folder(path):
    exists = os.path.exists(path)
    if not exists:
       os.makedirs(path)

In [28]:
def download_league_details(league_id): 
    url = f'https://draft.premierleague.com/api/league/{league_id}/details'
    response = requests.get(url)

    if response.status_code == 200:
        file_name = f'data/{league_id}/league_details.json'
    
        with open(file_name, "wb") as file:
            file.write(response.content)
    else: 
        raise Exception('Cant download results')

In [33]:
def download_elements(): 
    url = 'https://draft.premierleague.com/api/bootstrap-static'
    response = requests.get(url)

    if response.status_code == 200:
        file_name = f'data/bootstrap-static.json'
    
        with open(file_name, "wb") as file:
            file.write(response.content)
    else: 
        raise Exception('Cant download results')

In [34]:
def download_entry_picks(league_id, entry_id, week): 
    url = f'https://draft.premierleague.com/api/entry/{entry_id}/event/{week}'
    response = requests.get(url)

    if response.status_code == 200:
        file_name = f'data/{league_id}/entries/entry_{entry_id}_{week}_picks.json'
    
        with open(file_name, "wb") as file:
            file.write(response.content)
    else: 
        raise Exception('Cant download results')

In [38]:
def download_weekly_live_points(league_id, week): 
    url = f'https://draft.premierleague.com/api/event/{week}/live'
    response = requests.get(url)

    if response.status_code == 200:
        file_name = f'data/{league_id}/points/{week}_points.json'
    
        with open(file_name, "wb") as file:
            file.write(response.content)
    else: 
        raise Exception('Cant download results')

In [37]:
# Download shared bootstrap
create_folder('./data')
download_elements()

# Download league specific data
for league in leagues:
    create_folder(f'./data/{league}')
    create_folder(f'./data/{league}/entries')
    create_folder(f'./data/{league}/points')
    
    download_league_details(league)

    with open(f'data/{league}/league_details.json','r') as f:
        league_data = json.loads(f.read())

    teams = {}
    for team in league_data['league_entries']:
        id = team['entry_id']
        name = team['entry_name']
    
        if not id in teams:
            teams[id] = name
    
    for i in range(38):
        week = i+1
        download_weekly_live_points(league, week)
        for team in teams:
            download_entry_picks(league, team, week)